In [40]:
import sagemaker 
import boto3
from sagemaker import get_execution_role 

role = get_execution_role()
session = sagemaker.Session()
region_name = boto3.Session().region_name

In [41]:
%store -r s3_bucket
%store -r prefix

In [42]:
training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training.jsonl"
test_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/test.jsonl"
training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/"

In [43]:
from sagemaker.inputs import TrainingInput
train = TrainingInput(
    training_s3_input_location, 
    content_type="json"
)
test = TrainingInput(
    test_s3_input_location, 
    content_type="json"
)

In [44]:
%store -r prediction_length
prediction_length

50

In [45]:
%store -r freq
freq

'H'

In [46]:
context_length = prediction_length

In [47]:
from sagemaker.image_uris import retrieve 
container = retrieve(
    "forecasting-deepar", 
    region_name, 
    "1"
)
container

'522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1'

In [48]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.c4.2xlarge',
    output_path=training_s3_output_location,
    sagemaker_session=session
)

In [49]:
estimator.set_hyperparameters(
    time_freq=freq,
    context_length=str(context_length),
    prediction_length=str(prediction_length),
    num_cells=40,
    num_layers=3,
    likelihood="gaussian",
    epochs=20,
    mini_batch_size=32,
    learning_rate=0.001,
    dropout_rate=0.05,
    early_stopping_patience=10
)

In [50]:
data_channels = {"train": train, "test": test}

In [51]:
%%time

estimator.fit(inputs=data_channels)

2021-06-01 11:05:17 Starting - Starting the training job...
2021-06-01 11:05:18 Starting - Launching requested ML instancesProfilerReport-1622545517: InProgress
......
2021-06-01 11:06:44 Starting - Preparing the instances for training.........
2021-06-01 11:08:14 Downloading - Downloading input data
2021-06-01 11:08:14 Training - Downloading the training image...
2021-06-01 11:08:35 Training - Training image download completed. Training in progress.Arguments: train
[06/01/2021 11:08:39 INFO 139987855828352] Reading default configuration from /opt/amazon/lib/python3.6/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto', 'dropout_rate': '0.10', 'early_stopping_patience': '', 'embedding_dimension': '10', 'learning_rate': '0.001', 'likelihood': 'student-t', 'mini_batch_size': '128', 'num_cells': '40', 'num_dynamic_feat': 'auto', 'num_eval_samples': '100', 'num_layer

In [52]:
estimator.model_data

's3://sagemaker-cookbook-bucket/chapter08/output/forecasting-deepar-2021-06-01-11-05-17-028/output/model.tar.gz'

In [53]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge"
)

---------------!

In [54]:
endpoint_name = predictor.endpoint_name

In [55]:
%store endpoint_name

Stored 'endpoint_name' (str)
